In [ ]:
⎕IO ← 0            ⍝ Index origin is zero
]box on -style=max ⍝ Show boxes at max verbosity
]rows on           ⍝ Don't wrap long output lines
I←⌷⍨∘⊃⍨⍤0 99       ⍝ Sane indexing
assert ← {⍺ ← 'assertion failure' ⋄ 0∊⍵: ⍺ ⎕signal 8 ⋄ shy ← 0} ⍝ assert

## Part: visible trees

#### Example data

In [ ]:
⎕ ← heights ← ⊃⎕NGET'ex1.txt'1
⎕ ← heights ← ↑⍎¨¨heights

In [ ]:
⍝ The basic algorithm (from the internet)
⌈⍀heights                       ⍝ Max scan downwards
¯1⍪¯1↓⌈⍀heights                 ⍝ Drop the last row and catenate -1 because we can always see the first row
heights>¯1⍪¯1↓⌈⍀heights         ⍝ Compare the heights. 1 = visible
find_visible ← {⍵>¯1⍪¯1↓⌈⍀⍵}

In [ ]:
⍝ Use rotation to do it for all fours direction
4⍴⊂heights
{⌽⍉⍵}\4⍴⊂heights
rotations ← {{⌽⍉⍵}\4⍴⊂⍵}
find_visible¨{⌽⍉⍵}\4⍴⊂heights

In [ ]:
⍝ Need to move to the same orientation
rotations heights                ⍝ Rotate
↑rotations¨rotations heights     ⍝ Get the rotations for each rotation
(0 3)(1 2)(2 1)(3 0)I↑rotations¨rotations heights     ⍝ Pick out a set of the same rotations
get_same_orientation ← {(0 3)(1 2)(2 1)(3 0)I↑rotations¨⍵}
{{⌽⍉⍵}\⌽¨{⌽⍉⍵}\4⍴⊂⍵}heights     ⍝ Internet alternative - doesn't return to original orientation but all end up in the same orientation

In [ ]:
get_same_orientation find_visible¨rotations heights
∨⌿get_same_orientation find_visible¨rotations heights                 ⍝ Reduce using 'or'
,⊃∨⌿get_same_orientation find_visible¨rotations heights               ⍝ Flatten
+/,⊃∨⌿get_same_orientation find_visible¨rotations heights             ⍝ Count trees

### Real data

In [ ]:
heights ← ↑⍎¨¨⊃⎕NGET'input.txt'1
assert 1647 = +/,⊃∨⌿get_same_orientation find_visible¨rotations heights

In [ ]:
⍝ Internet solution
h←⍎¨↑⊃⎕nget'input.txt'1
f←{↑{⌽⍉⍵}\⌽⍺⍺¨{⌽⍉⍵}\4⍴⊂⍵}
assert 1647 = +/,∨⌿(⊢>¯1⍪¯1↓⌈⍀)f h

## Part 2: furthest view from a tree

In [ ]:
⎕ ← heights ← ⍎¨↑⊃⎕nget'ex1.txt'1

In [ ]:
,⍀heights

In [ ]:
heights{⍺>⍵}¨,⍀heights